# Efficiency and Diversity of R&D in Knowledge‑Intensive Services (2005‑2023)

## Introduction

The knowledge‑intensive services sector ('G‑N' sector in NACE classification, list available: __[LINK](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:High-tech_classification_of_manufacturing_industries)__) – including wholesale & retail trade, transportation, information & communication, finance, professional activities and administrative services – has become a major engine of innovation in Europe. 

A growing body of confessions suggests that gender diversity within research and development (R&D) teams enhances creativity and innovation. As Scientific American reports, diverse groups “are more innovative, more diligent, and better at solving complex problems” because they are able to produce and apply different perspectives (Phillips, 2014). Furthermore, firms with greater gender diversity tend to achieve higher productivity and innovation performance, particularly in knowledge-intensive sectors where collaboration and problem-solving are key (Hoogendoorn et al., 2019). The business press echoes this view — Forbes (2024) reports that companies introducing diversity consistently outperform rivals. 

The goal is to analyse how efficiently countries from the EU and EFTA convert R&D spending in the knowledge‑intensive services sector into human capital and to examine whether increasing female participation correlates with improved efficiency and labour intensity. Relative shares and growth rates over time will be central to the analysis.

**External data sources:**

- Business enterprise R&D expenditure in high-tech sectors by NACE Rev. 2 __[LINK: htec_sti_exp2](https://ec.europa.eu/eurostat/databrowser/view/htec_sti_exp2/default/table)__

- Business enterprise R&D personnel in high-tech sectors by NACE Rev. 2 __[LINK: htec_sti_pers2](https://ec.europa.eu/eurostat/databrowser/view/htec_sti_pers2/default/table)__

- R&D personnel and researchers in business enterprise sector by NACE Rev. 2 activity and sex __[LINK: rd_p_bempoccr2](https://db.nomics.world/Eurostat/rd_p_bempoccr2?dimensions=%7B%22freq%22%3A%5B%22A%22%5D%2C%22nace_r2%22%3A%5B%22G-N%22%5D%7D&tab=table)__ 

**Objectives:** 

- O1 Extract Eurostat datasets and metadata.

- O2 Pre-process the datasets: clean and transform datasets, unify variable names.

- O3 Merge datasets and save them in the .csv format.

File metadata:

In [1]:
# __author__ = Dominika Drazyk
# __maintainer__ = Dominika Drazyk
# __email__ = dominika.a.drazyk@gmail.com
# __copyright__ = Dominika Drazyk
# __license__ = Apache License 2.0
# __version__ = 1.0.0
# __status__ = Production
# __date__ = 30/09/2025

Required libraries:

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from pyjstat import pyjstat
from datetime import datetime
import pandas as pd
import requests
import time
import re
import os

## O1 Data Scraping Pipeline

### O1.1 Extracting Eurostat datasets

Extracting three source datasets from Eurostat using API calls and data transformation techniques.

Eurostat provides API access to extract JSON-formatted datasets. The following code extracts expenditure and personnel data, transforming JSON format into pandas DataFrames for further processing.

In [3]:
print("• Extracting expenditure data (htec_sti_exp2)")
url_exp2 = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/htec_sti_exp2?lang=en"
response_exp2 = requests.get(url_exp2)
response_exp2.raise_for_status()
data_exp2 = response_exp2.json()
data_exp2 = pyjstat.from_json_stat(data_exp2, naming = 'id')[0]
print(f"✓ Expenditure data extracted: {len(data_exp2):,} records")
print(f"  Sample data: {data_exp2.shape}")
print(data_exp2.head())

• Extracting expenditure data (htec_sti_exp2)
✓ Expenditure data extracted: 11,970 records
  Sample data: (11970, 6)
  freq     unit nace_r2        geo  time       value
0    A  MIO_EUR   TOTAL  EU27_2020  2005  107641.827
1    A  MIO_EUR   TOTAL  EU27_2020  2006  116175.251
2    A  MIO_EUR   TOTAL  EU27_2020  2007  123195.979
3    A  MIO_EUR   TOTAL  EU27_2020  2008  131732.974
4    A  MIO_EUR   TOTAL  EU27_2020  2009  129092.773


In [4]:
print("• Extracting personnel data (htec_sti_pers2)")
url_pers2 = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/htec_sti_pers2?lang=en"
response_pers2 = requests.get(url_pers2)
response_pers2.raise_for_status()
data_pers2 = response_pers2.json()
data_pers2 = pyjstat.from_json_stat(data_pers2, naming = 'id')[0]
print(f"✓ Personnel data extracted: {len(data_pers2):,} records")
print(f"  Sample data: {data_pers2.shape}")
print(data_pers2.head())

• Extracting personnel data (htec_sti_pers2)
✓ Personnel data extracted: 23,940 records
  Sample data: (23940, 7)
  freq nace_r2 unit prof_pos        geo  time      value
0    A   TOTAL  FTE    TOTAL  EU27_2020  2005   982304.1
1    A   TOTAL  FTE    TOTAL  EU27_2020  2006  1036105.4
2    A   TOTAL  FTE    TOTAL  EU27_2020  2007  1077303.3
3    A   TOTAL  FTE    TOTAL  EU27_2020  2008  1130735.5
4    A   TOTAL  FTE    TOTAL  EU27_2020  2009  1123274.0


The female researcher dataset requires pre-filtering and is available as a CSV file. The dataset is transformed after loading to match the structure of the other two datasets for consistent processing.

In [5]:
print("• Extracting female researcher data")
url_fem2 = "https://db.nomics.world/Eurostat/rd_p_bempoccr2?dimensions=%7B%22freq%22%3A%5B%22A%22%5D%2C%22nace_r2%22%3A%5B%22G-N%22%5D%7D&tab=table"
data_fem2 = pd.read_csv('../data/rd_p_bempoccr2.csv')

# Process column names
for col in data_fem2.columns:
    if col.startswith('Annual'):
        match = re.search(r'\.([A-Z]{2,3})\)$', col)
        if match and match.group(1):
          geo = match.group(1)
        else:
          geo = 'UU'
        data_fem2 = data_fem2.rename(columns={col: geo})
        
# Reshape data
data_fem2 = data_fem2.melt(id_vars = ['period'], 
                           value_vars = data_fem2.columns[1:274], 
                           var_name = 'geo', 
                           value_name = 'fem2_FTE_RSE')
data_fem2 = data_fem2[data_fem2['geo'] != "UU"]
data_fem2 = data_fem2.rename(columns={'period': 'time'})
data_fem2['time'] = data_fem2['time'].map(str)
data_fem2['nace_r2'] = 'G-N'
print(f"✓ Female researcher data processed: {len(data_fem2):,} records")
print(f"  Sample data: {data_fem2.shape}")
print()
print(data_fem2.head())

• Extracting female researcher data
✓ Female researcher data processed: 4,769 records
  Sample data: (4769, 4)

   time geo  fem2_FTE_RSE nace_r2
0  2005  AT           NaN     G-N
1  2006  AT         999.2     G-N
2  2007  AT        1110.0     G-N
3  2008  AT           NaN     G-N
4  2009  AT        1829.8     G-N


### O1.2 Extracting dataset metadata

Extract dataset metadata including last update timestamps, data sources, titles, and dataset identifiers from source webpages for documentation and traceability.

Each source webpage contains metadata that is automatically updated upon data refresh.

In [6]:
print("• Extracting expenditure dataset metadata")
url_exp2_meta = 'https://ec.europa.eu/eurostat/databrowser/view/htec_sti_exp2/default/table'
print(f"  Source: {url_exp2_meta}")

chrome_options = Options()
driver_exp2 = webdriver.Chrome(options = chrome_options)
driver_exp2.get(url_exp2_meta)
print("  • Webpage opened")
time.sleep(20) 
r = driver_exp2.page_source
print("  • Page source extracted")
soup_exp2 = bs(r, "html.parser")
driver_exp2.close()
print("  • Browser closed")

• Extracting expenditure dataset metadata
  Source: https://ec.europa.eu/eurostat/databrowser/view/htec_sti_exp2/default/table
  • Webpage opened
  • Page source extracted
  • Browser closed


In [7]:
print("  • Parsing metadata fields")
body = soup_exp2.find('body')
marker = body.find('span', string = "last update")
tag = marker.find_next("b", class_ = "infobox-text-data")
exp2_date = tag.get_text(strip = True)
print(f"    - Last updated: {exp2_date}")

marker = body.find('span', string = "Source of data:")
tag = marker.find_next("span")
exp2_source = tag.get_text(strip = True)
print(f"    - Source: {exp2_source}")

exp2_title = soup_exp2.find('h1', class_ = "ecl-page-header__title").get_text()
print(f"    - Title: {exp2_title}")

marker = body.find('span', string = "Online data code:")
tag = marker.find_next("b", class_ = "infobox-text-data")
exp2_id = tag.get_text(strip = True)
print(f"    - Dataset ID: {exp2_id}")

exp2_meta = [exp2_id, exp2_source, exp2_title, exp2_date]

  • Parsing metadata fields
    - Last updated: 29/09/2025 23:00
    - Source: Eurostat
    - Title: Business enterprise R&D expenditure in high-tech sectors by NACE Rev. 2
    - Dataset ID: htec_sti_exp2


In [8]:
print("• Extracting personnel dataset metadata")
url_pers2_meta = 'https://ec.europa.eu/eurostat/databrowser/view/htec_sti_pers2/default/table'
print(f"  Source: {url_pers2_meta}")

chrome_options = Options()
driver_pers2 = webdriver.Chrome(options = chrome_options)
driver_pers2.get(url_pers2_meta)
print("  • Webpage opened")
time.sleep(20) 
r = driver_pers2.page_source
print("  • Page source extracted")
soup_pers2 = bs(r, "html.parser")
driver_pers2.close()
print("  • Browser closed")

• Extracting personnel dataset metadata
  Source: https://ec.europa.eu/eurostat/databrowser/view/htec_sti_pers2/default/table
  • Webpage opened
  • Page source extracted
  • Browser closed


In [9]:
print("  • Parsing metadata fields")
body = soup_pers2.find('body')
marker = body.find('span', string = "last update")
tag = marker.find_next("b", class_ = "infobox-text-data")
pers2_date = tag.get_text(strip = True)
print(f"    - Last updated: {pers2_date}")

marker = body.find('span', string = "Source of data:")
tag = marker.find_next("span")
pers2_source = tag.get_text(strip = True)
print(f"    - Source: {pers2_source}")

pers2_title = soup_pers2.find('h1', class_ = "ecl-page-header__title").get_text()
print(f"    - Title: {pers2_title}")

marker = body.find('span', string = "Online data code:")
tag = marker.find_next("b", class_ = "infobox-text-data")
pers2_id = tag.get_text(strip = True)
print(f"    - Dataset ID: {pers2_id}")

pers2_meta = [pers2_id, pers2_source, pers2_title, pers2_date]

  • Parsing metadata fields
    - Last updated: 29/09/2025 23:00
    - Source: Eurostat
    - Title: Business enterprise R&D personnel in high-tech sectors by NACE Rev. 2
    - Dataset ID: htec_sti_pers2


In [10]:
print("• Extracting female researcher dataset metadata")
url_fem2_meta = 'https://db.nomics.world/Eurostat/rd_p_bempoccr2?dimensions=%7B%22freq%22%3A%5B%22A%22%5D%2C%22nace_r2%22%3A%5B%22G-N%22%5D%7D&tab=table'
print(f"  Source: {url_fem2_meta}")

chrome_options = Options()
driver_fem2 = webdriver.Chrome(options = chrome_options)
driver_fem2.get(url_fem2_meta)
print("  • Webpage opened")
time.sleep(20) 
r = driver_fem2.page_source
print("  • Page source extracted")
soup_fem2 = bs(r, "html.parser")
driver_fem2.close()
print("  • Browser closed")

• Extracting female researcher dataset metadata
  Source: https://db.nomics.world/Eurostat/rd_p_bempoccr2?dimensions=%7B%22freq%22%3A%5B%22A%22%5D%2C%22nace_r2%22%3A%5B%22G-N%22%5D%7D&tab=table
  • Webpage opened
  • Page source extracted
  • Browser closed


In [11]:
print("  • Parsing metadata fields")
body = soup_fem2.find('body')
marker = body.find("p", class_ = "my-8")
text = marker.get_text(strip = True)
match = re.search(r'on(\w+\s+\d+,\s+\d+)\s+\((\d+:\d+\s+[AP]M)\)', text)
if match:
    date_str = match.group(1)
    time_str = match.group(2)
    dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
    fem2_date = dt.strftime("%d/%m/%Y %H:%M")
    print(f"    - Last updated: {fem2_date}")
else: 
    fem2_date = 'None'
    print(f"    - Last updated: {fem2_date}")

div = body.find('div', class_ = "container")
span = div.find('span', class_ = "hover:text-foreground transition-colors")
a = span.find_next('a', class_ = "text-muted-foreground link")
fem2_source = a.get_text(strip = True)[1:-1]
print(f"    - Source: {fem2_source}")

div = body.find('div', class_ = "container")
h1 = div.find('h1', class_ = "text-3xl mb-10")
spans = h1.find_all('span')
fem2_title = spans[3].get_text(strip = True)
print(f"    - Title: {fem2_title}")

marker = h1.find('span', class_ = "text-muted-foreground")
fem2_id = marker.get_text(strip = True)[1:-1]
print(f"    - Dataset ID: {fem2_id}")

fem2_meta = [fem2_id, fem2_source, fem2_title, fem2_date]

  • Parsing metadata fields
    - Last updated: 02/05/2025 11:00
    - Source: Eurostat
    - Title: R&D personnel and researchers in business enterprise sector by NACE Rev. 2 activity and sex
    - Dataset ID: rd_​p_​bempoccr2


In [12]:
print("• Creating metadata dataset")
meta = pd.DataFrame([exp2_meta, pers2_meta, fem2_meta], 
                   columns = ['dataset_id', 'dataset_source', 'dataset_title', 'dataset_last_updated'])
print(f"✓ Metadata dataset created: {meta.shape[0]} datasets")
print(meta)

• Creating metadata dataset
✓ Metadata dataset created: 3 datasets
         dataset_id dataset_source  \
0     htec_sti_exp2       Eurostat   
1    htec_sti_pers2       Eurostat   
2  rd_​p_​bempoccr2       Eurostat   

                                       dataset_title dataset_last_updated  
0  Business enterprise R&D expenditure in high-te...     29/09/2025 23:00  
1  Business enterprise R&D personnel in high-tech...     29/09/2025 23:00  
2  R&D personnel and researchers in business ente...     02/05/2025 11:00  


In [13]:
print("• Saving metadata")
meta.to_csv('../data/scraper_metadata.csv', encoding='utf-8', index = False)
print("✓ Metadata saved: ../data/scraper_metadata.csv")
print()

• Saving metadata
✓ Metadata saved: ../data/scraper_metadata.csv



### O1.3 Extracting EU + EFTA countries list

Extract the list of European Union and European Free Trade Association (EFTA) countries from official Eurostat documentation for filtering.

In [14]:
url_countries_meta = 'https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Country_codes'
print(f"• Source: {url_countries_meta}")

chrome_options = Options()
driver_co = webdriver.Chrome(options = chrome_options)
driver_co.get(url_countries_meta)
print("  • Webpage opened")
time.sleep(20) 
r = driver_co.page_source
print("  • Page source extracted")
soup_co = bs(r, "html.parser")
driver_co.close()
print("  • Browser closed")

• Source: https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Country_codes
  • Webpage opened
  • Page source extracted
  • Browser closed


In [15]:
print("  • Parsing country data from tables")
content_div = soup_co.find('div', {'id': 'mw-content-text'})
tables = content_div.find_all('table')
eu_efta_countries = []

for i, table in enumerate(tables):
    if (i == 0) or (i == 1):  
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            for j in range(0, len(cells), 2):
                if j + 1 < len(cells):
                    if cells[j].get_text(strip=True) == '':
                        j = j + 1
                    country_name = cells[j].get_text(strip=True)
                    country_code = cells[j + 1].get_text(strip=True)
                    country_code = country_code.replace('(', '').replace(')', '').strip()
                    print(f"    - {country_name}: {country_code}")

                    country_data = {'Country': country_name, 'geo': country_code}
                    eu_efta_countries.append(country_data)

eu_efta_countries_df = pd.DataFrame.from_dict(eu_efta_countries)
print(f"✓ Countries extracted: {len(eu_efta_countries_df)} countries")

  • Parsing country data from tables
    - Belgium: BE
    - Greece: EL
    - Lithuania: LT
    - Portugal: PT
    - Bulgaria: BG
    - Spain: ES
    - Luxembourg: LU
    - Romania: RO
    - Czechia: CZ
    - France: FR
    - Hungary: HU
    - Slovenia: SI
    - Denmark: DK
    - Croatia: HR
    - Malta: MT
    - Slovakia: SK
    - Germany: DE
    - Italy: IT
    - Netherlands: NL
    - Finland: FI
    - Estonia: EE
    - Cyprus: CY
    - Austria: AT
    - Sweden: SE
    - Ireland: IE
    - Latvia: LV
    - Poland: PL
    - Iceland: IS
    - Norway: NO
    - Liechtenstein: LI
    - Switzerland: CH
✓ Countries extracted: 31 countries


In [16]:
print("• Saving countries list")
eu_efta_countries_df.to_csv('../data/eu_efta_countries.csv', encoding = 'utf-8', index = False)
print("✓ Countries saved: ../data/eu_efta_countries.csv")
print()

• Saving countries list
✓ Countries saved: ../data/eu_efta_countries.csv



## O2 Preprocessing datasets

This section removes unnecessary columns, and transforms datasets from long to wide format to prepare for merging and analysis.

In [17]:
print(data_exp2.info())
print(data_pers2.info())
print(data_fem2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11970 entries, 0 to 11969
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   freq     11970 non-null  object 
 1   unit     11970 non-null  object 
 2   nace_r2  11970 non-null  object 
 3   geo      11970 non-null  object 
 4   time     11970 non-null  object 
 5   value    7262 non-null   float64
dtypes: float64(1), object(5)
memory usage: 561.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23940 entries, 0 to 23939
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   freq      23940 non-null  object 
 1   nace_r2   23940 non-null  object 
 2   unit      23940 non-null  object 
 3   prof_pos  23940 non-null  object 
 4   geo       23940 non-null  object 
 5   time      23940 non-null  object 
 6   value     12908 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1.3+ MB
None
<class 'panda

In [18]:
print(data_exp2.nunique())
print(data_pers2.nunique())
print(data_fem2.nunique())

freq          1
unit          2
nace_r2       7
geo          45
time         19
value      5739
dtype: int64
freq            1
nace_r2         7
unit            2
prof_pos        2
geo            45
time           19
value       10163
dtype: int64
time              19
geo               38
fem2_FTE_RSE    2685
nace_r2            1
dtype: int64


Both expenditure and personnel datasets contain a categorical variable *freq* with only one level. This variable is removed during data cleaning as it provides no analytical value. 

In [19]:
print("• Cleaning datasets")
data_exp2.drop(columns = ['freq'], inplace = True)
data_pers2.drop(columns = ['freq'], inplace = True) 
print("  ✓ Unused columns removed")

• Cleaning datasets
  ✓ Unused columns removed


The expenditure and personnel datasets share common categorical variables (*nace_r2*, *geo*, *time*) that serve as merge keys. The remaining categorical variables (*unit* for expenditure data; *unit* and *prof_pos* for personnel data) represent different metrics stored in the *value* column. These variables are transformed into separate columns (*exp2_MIO_EUR*, *exp2_PC_TOT*, *pers2_FTE_RSE*, *pers2_FTE_TOTAL*, *pers2_HC_RSE*, *pers2_HC_TOTAL*) through pivot operations to enable proper dataset merging.

In [20]:
print('Dataset exp2, \'unit\' levels  : ', data_exp2['unit'].unique().tolist())
print('Dataset pers2, \'unit\' levels : ', data_pers2['unit'].unique().tolist(), '\n')
print('Dataset pers2, \'prof_pos\' levels : ', data_pers2['prof_pos'].unique().tolist(), '\n')
print('Dataset exp2, \'nace_r2\' levels  : ', data_exp2['nace_r2'].unique().tolist())
print('Dataset pers2, \'nace_r2\' levels : ', data_pers2['nace_r2'].unique().tolist())
print('Dataset fem2, \'nace_r2\' levels : ', data_fem2['nace_r2'].unique().tolist(), '\n')
print('Dataset exp2, \'geo\' levels  : ', data_exp2['geo'].unique().tolist())
print('Dataset pers2, \'geo\' levels : ', data_pers2['geo'].unique().tolist())
print('Dataset fem2, \'geo\' levels : ', data_fem2['geo'].unique().tolist(), '\n')
print('Dataset exp2, \'time\' levels  : ', data_exp2['time'].unique().tolist())
print('Dataset pers2, \'time\' levels : ', data_pers2['time'].unique().tolist())
print('Dataset fem2, \'time\' levels : ', data_fem2['time'].unique().tolist())

Dataset exp2, 'unit' levels  :  ['MIO_EUR', 'PC_TOT']
Dataset pers2, 'unit' levels :  ['FTE', 'HC'] 

Dataset pers2, 'prof_pos' levels :  ['TOTAL', 'RSE'] 

Dataset exp2, 'nace_r2' levels  :  ['TOTAL', 'C', 'C_HTC_M', 'C_HTC', 'C_LTC_M', 'C_LTC', 'G-N']
Dataset pers2, 'nace_r2' levels :  ['TOTAL', 'C', 'C_HTC_M', 'C_HTC', 'C_LTC_M', 'C_LTC', 'G-N']
Dataset fem2, 'nace_r2' levels :  ['G-N'] 

Dataset exp2, 'geo' levels  :  ['EU27_2020', 'EA20', 'EA19', 'BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE', 'IS', 'NO', 'CH', 'UK', 'BA', 'ME', 'MK', 'AL', 'RS', 'TR', 'RU', 'US', 'CN_X_HK', 'JP', 'KR']
Dataset pers2, 'geo' levels :  ['EU27_2020', 'EA20', 'EA19', 'BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE', 'IS', 'NO', 'CH', 'UK', 'BA', 'ME', 'MK', 'AL', 'RS', 'TR', 'RU'

In [21]:
print("• Transforming expenditure data to wide format")
data_exp2_wide = data_exp2.pivot(index = ['nace_r2', 'geo', 'time'], columns = 'unit', values = 'value').reset_index()
data_exp2_wide = data_exp2_wide.rename(columns=lambda x: f"exp2_{x}" if x not in ['nace_r2', 'geo', 'time'] else x)
print(f"  ✓ Expenditure data: {data_exp2_wide.shape[0]:,} rows × {data_exp2_wide.shape[1]} columns")
print(f"    Sample: {data_exp2_wide.shape}")
data_exp2_wide.head()

• Transforming expenditure data to wide format
  ✓ Expenditure data: 5,985 rows × 5 columns
    Sample: (5985, 5)


unit,nace_r2,geo,time,exp2_MIO_EUR,exp2_PC_TOT
0,C,AL,2005,NaN,NaN
1,C,AL,2006,NaN,NaN
2,C,AL,2007,NaN,NaN
3,C,AL,2008,NaN,NaN
4,C,AL,2009,NaN,NaN


In [23]:
print("• Transforming personnel data to wide format")
data_pers2_wide = data_pers2.pivot(index = ['nace_r2', 'geo', 'time', 'prof_pos'], 
                                   columns = ['unit'], 
                                   values = 'value').reset_index()
data_pers2_wide = data_pers2_wide.pivot(index = ['nace_r2', 'geo', 'time'], 
                                        columns = ['prof_pos'], 
                                        values = ['FTE','HC']).reset_index()
data_pers2_wide.columns = ["_".join([str(c) for c in col if c != ""])
                            for col in data_pers2_wide.columns.to_flat_index()]
data_pers2_wide = data_pers2_wide.rename(columns=lambda x: f"pers2_{x}" 
                                         if x not in ['nace_r2', 'geo', 'time'] else x)
print(f"  ✓ Personnel data: {data_pers2_wide.shape[0]:,} rows × {data_pers2_wide.shape[1]} columns")
print(f"    Sample: {data_pers2_wide.shape}")
data_pers2_wide.head()

• Transforming personnel data to wide format
  ✓ Personnel data: 5,985 rows × 7 columns
    Sample: (5985, 7)


,nace_r2,geo,time,pers2_FTE_RSE,pers2_FTE_TOTAL,pers2_HC_RSE,pers2_HC_TOTAL
0,C,AL,2005,NaN,NaN,NaN,NaN
1,C,AL,2006,NaN,NaN,NaN,NaN
2,C,AL,2007,NaN,NaN,NaN,NaN
3,C,AL,2008,NaN,NaN,NaN,NaN
4,C,AL,2009,NaN,NaN,NaN,NaN


## O3 Merging datasets

This section merges the preprocessed datasets using common keys and exports the final consolidated dataset for further analysis.

In [24]:
print("• Merging all datasets")
data = pd.merge(data_pers2_wide, data_exp2_wide, on = ['nace_r2', 'geo', 'time'], how = 'left') 
data = pd.merge(data, data_fem2, on = ['nace_r2', 'geo', 'time'], how = 'left') 
print(f"✓ Merged dataset: {data.shape[0]:,} rows × {data.shape[1]} columns")
print(f"  Sample data: {data.shape}")

print("• Dataset summary:")
print(f"  - NACE levels: {data['nace_r2'].unique().tolist()}")
print(f"  - Geographic levels: {len(data['geo'].unique())} countries")
print(f"  - Time levels: {len(data['time'].unique())} years")
data.sample(10)

• Merging all datasets
✓ Merged dataset: 10,032 rows × 10 columns
  Sample data: (10032, 10)
• Dataset summary:
  - NACE levels: ['C', 'C_HTC', 'C_HTC_M', 'C_LTC', 'C_LTC_M', 'G-N', 'TOTAL']
  - Geographic levels: 45 countries
  - Time levels: 19 years


,nace_r2,geo,time,pers2_FTE_RSE,pers2_FTE_TOTAL,pers2_HC_RSE,pers2_HC_TOTAL,exp2_MIO_EUR,exp2_PC_TOT,fem2_FTE_RSE
7156,G-N,LU,2017,747.8,1368.8,941.0,1942.0,157.400,39.15,941.0
4835,G-N,CH,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3425,C_LTC_M,AL,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5170,G-N,CZ,2016,10335.6,17866.9,12338.0,23409.0,796.654,43.97,1864.0
4784,G-N,BG,2018,4768.0,7324.0,5579.0,8603.0,198.116,65.00,1630.0
832,C,UK,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,C,ME,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,C_HTC_M,HU,2011,2220.0,3122.0,2460.0,3613.0,143.401,19.07,NaN
8204,G-N,PT,2020,13224.2,17949.0,24344.0,33291.0,1064.027,57.72,17949.0
9086,G-N,UK,2012,50065.0,87618.7,NaN,NaN,11955.061,56.67,NaN


In [25]:
print("• Saving merged dataset")
data.to_csv('../data/scraper_data.csv', encoding='utf-8', index = False)
print("✓ Merged dataset saved: ../data/scraper_data.csv")
print()

• Saving merged dataset
✓ Merged dataset saved: ../data/scraper_data.csv

